In [19]:
# Mission to Mars
# By: Chris Estrada
# https://chrisestrada007.github.io/

# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
import pymongo

In [20]:
# Scrape Nasa Mars Site
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
headline_soup = bs(response.text, 'lxml')
# Get Headline
headline = headline_soup.find('div', class_ = 'content_title').text.strip()
print(headline)
# Get 1st paragraph
news = headline_soup.find('div', class_ = 'rollover_description_inner').text.strip()
print(news)

NASA's Curiosity Mars Rover Finds a Clay Cache
The rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.


In [21]:
# JPL Mars Space Images - Featured Image
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

# Parse HTML with Beautiful Soup
image_soup = bs(browser.html, 'html.parser')
# Retrieve background-image url from style tag 
featured_image_url  = image_soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
# Website Url 
main_url = 'https://www.jpl.nasa.gov'
# Concatenate website url with scrapped route
featured_image_url = main_url + featured_image_url
# Display full link to featured image
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18904-1920x1200.jpg'

In [22]:
# Mars Weather
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
weather_soup = bs(browser.html, 'html.parser')

# Find all tweets
latest_tweets = weather_soup.find_all('div', class_='js-tweet-text-container')

# find the first weather tweet (Sol)
for tweet in latest_tweets: 
    weather_tweet = tweet.find('p').text
    if 'Sol' and 'pressure' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass

InSight sol 178 (2019-05-28) low -101.1ºC (-150.0ºF) high -23.0ºC (-9.5ºF)
winds from the SW at 4.8 m/s (10.8 mph) gusting to 15.0 m/s (33.5 mph)
pressure at 7.50 hPapic.twitter.com/4Ejlnu9Kam


In [23]:
# Mars Facts
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

# Pandas to HTML
mars_df = pd.read_html(facts_url)
mars_df = (mars_df[0])
mars_df.columns = ["Description", "Value"]
mars_df = mars_df.set_index("Description")
mars_facts_html = mars_df.to_html(header = False, index = False)
print(mars_facts_html)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [24]:
# Mars Hemispheres
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)
html_hemi = browser.html

# Soup it
hemi_soup = bs(html_hemi, 'html.parser')
items = hemi_soup.find_all('div', class_='item')
# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = bs(partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]